<a href="https://colab.research.google.com/github/sttteephen/Cellular-Automata/blob/main/coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Split Data

In [3]:
# imports and mount google drive
import pandas as pd
from keras.models import Sequential
from keras import layers
from google.colab import drive
import nltk
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras import preprocessing
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np

drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# create dataframe from dataset and split into reviews and labels
df_train = pd.read_csv('/content/drive/My Drive/coursework_dataset/all_data.csv', names=['reviews', 'labels'], sep=',')
reviews = df_train['reviews'].values
labels = df_train['labels'].values


In [5]:
# create smaller dataframe from smaller dataset for exploratory analysis, split into reveiws and labels
df_analysis = pd.read_csv('/content/drive/My Drive/coursework_dataset/train.csv', names=['reviews', 'labels'], sep=',')
a_reviews = df_analysis['reviews'].values
a_labels = df_analysis['labels'].values

In [6]:
len(reviews), len(a_reviews)

(2000, 1400)

# Explore Data

In [7]:
# get total review count and positive and negative review counts
total_review_count = len(reviews)
positive_review_count = len([x for x in labels if x == 1])
negative_review_count = len([x for x in labels if x == 0])

# determine minimum and maximumtext length
minimum_text_length = min([len(x) for x in list(reviews)])
maximum_text_length = max([len(x) for x in list(reviews)])

print(f'min review length (chars) - {minimum_text_length}\nmax review length (chars) - {maximum_text_length}')
print(f'total count - {total_review_count}\npositive count - {positive_review_count}\nnegative count - {negative_review_count}')

min review length (chars) - 91
max review length (chars) - 14957
total count - 2000
positive count - 1000
negative count - 1000


In [8]:
# separate positive and negative reviews
positive_reviews = [reviews[i] for i in range(len(reviews)) if labels[i] == 1]
negative_reviews = [reviews[i] for i in range(len(reviews)) if labels[i] == 0]


nltk.download('stopwords')
from nltk.corpus import stopwords

stopset = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# ngram frequency
positive_ngrams = []
negative_ngrams = []

n = 1

# only get ngrams that contain alph numeric tokens that are not in the stop set
for process in [(positive_reviews, positive_ngrams), (negative_reviews, negative_ngrams)]:
  for rev in process[0]:
    for ng in list(nltk.ngrams(rev.split(), n)):
      if False not in [x.isalnum() for x in ng] and len([x for x in ng if x not in stopset]) == n:
        process[1].append(ng)

pos_frequencies = nltk.FreqDist(positive_ngrams)
neg_frequencies = nltk.FreqDist(negative_ngrams)

print('positive', pos_frequencies.most_common(50))
print('negative', neg_frequencies.most_common(50))

positive [(('film',), 4857), (('one',), 2903), (('movie',), 2385), (('like',), 1712), (('story',), 1207), (('also',), 1200), (('good',), 1189), (('even',), 1174), (('time',), 1170), (('much',), 1026), (('character',), 1013), (('would',), 992), (('life',), 980), (('first',), 962), (('two',), 959), (('well',), 956), (('characters',), 953), (('see',), 951), (('way',), 915), (('get',), 883), (('films',), 872), (('best',), 807), (('many',), 780), (('really',), 775), (('make',), 772), (('little',), 768), (('people',), 767), (('great',), 743), (('new',), 718), (('never',), 716), (('scene',), 714), (('man',), 695), (('love',), 644), (('scenes',), 632), (('could',), 627), (('movies',), 627), (('world',), 619), (('still',), 585), (('us',), 581), (('plot',), 574), (('know',), 569), (('however',), 566), (('makes',), 560), (('another',), 559), (('back',), 557), (('performance',), 549), (('go',), 542), (('seen',), 537), (('seems',), 530), (('something',), 526)]
negative [(('film',), 3992), (('movie'

In [10]:
# average word length of reviews
def avg_word_length(rev):
    distinct_word_lengths = [len(w) for w in set(v.lower() for v in rev.split())]
    avg_word_length = float(sum(distinct_word_lengths)) / len(distinct_word_lengths)
    return avg_word_length


avg_pos_rev_len = sum([avg_word_length(r) for r in positive_reviews]) / len(positive_reviews)
avg_neg_rev_len = sum([avg_word_length(r) for r in negative_reviews]) / len(negative_reviews)
avg_pos_rev_len, avg_neg_rev_len

(5.69634338931218, 5.563958402025013)

In [11]:
def rev_len(rev):

    w_list = rev.split()
    total_words = len(w_list)

    distinct_w_list = []
    total_distinct_words = 0

    for w in w_list:
      lw = w.lower()
      if lw not in distinct_w_list:
        distinct_w_list.append(lw)
        total_distinct_words += 1
        #print(w)


    return (total_words, total_distinct_words)

avg_pos_rev_len = sum([rev_len(x)[0] for x in positive_reviews]) / len(positive_reviews)
avg_neg_rev_len = sum([rev_len(x)[0] for x in negative_reviews]) / len(negative_reviews)

pos_rev_dist_len = sum([rev_len(x)[1] for x in positive_reviews]) / len(positive_reviews)
neg_rev_dist_len = sum([rev_len(x)[1] for x in negative_reviews]) / len(negative_reviews)

print(avg_pos_rev_len, avg_neg_rev_len, pos_rev_dist_len, neg_rev_dist_len)

787.051 705.63 358.53 333.46


# Analyse Model Parameters

In [12]:
# produces a confusion matrix and determines Precision, Recall, Accuracy and F1-score of the given model

def eval_model(model):
  loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
  print("Testing Accuracy: {:.4f}".format(accuracy))

  # predict test data using model
  y_pred = model.predict(X_test, batch_size=50, verbose=1)
  y_pred_bool = []

  # convert probablities to classes
  for p in y_pred:
    if p >= 0.5:
      y_pred_bool.append(1)
    else:
      y_pred_bool.append(0)

  # print results
  cm = confusion_matrix(y_test, y_pred_bool)
  cf = classification_report(y_test, y_pred_bool)

  print('\t  Predicted')
  print('\t   0    1  ')
  print(f'\t0 {cm[0]}')
  print('True\t')
  print(f'\t1 {cm[1]}')

  print(cf)

  return (accuracy, cm, cf)

In [13]:
# pre-process testing data
def analysis_data():
  # split data into training and testing
  reviews_train, reviews_test, y_train, y_test = train_test_split(a_reviews, a_labels, test_size=0.20)

  # tokenize the data 
  tokenizer = Tokenizer(num_words=43297)
  tokenizer.fit_on_texts(reviews)
  X_train = tokenizer.texts_to_sequences(reviews_train)
  X_test = tokenizer.texts_to_sequences(reviews_test)

  vocab_size = len(tokenizer.word_index) + 1

  # the max length of a reviews, parameter for the embedding layer
  maxlen = max([len(x) for x in X_train] + [len(x) for x in X_test])

  # pad the tokenized data
  X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
  X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

  return (vocab_size, maxlen, X_train, X_test, y_train, y_test)

vocab_size, maxlen, X_train, X_test, y_train, y_test = analysis_data()
print(vocab_size, maxlen, len(X_train))

43297 2392 1120


In [ ]:
# test different numbers of LSTM layers

results = []

def lstm_layers_1():
  embedding_dim = 10
  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

def lstm_layers_2():
  embedding_dim = 10
  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

def lstm_layers_3():
  embedding_dim = 10
  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

models = [lstm_layers_1, lstm_layers_2, lstm_layers_3]
avg_acc = []
tests = 10

for model_func in models:

  total_accuracy = 0

  for i in range(tests):
    model = model_func()
    training = model.fit(X_train, y_train, epochs=5, verbose=True, validation_split=0.1, batch_size=50)
    accuracy, cm, cf = eval_model(model)

    print(accuracy)

    total_accuracy += accuracy

  mean_accuracy = total_accuracy / tests
  print('MEAN ACCURACY -', mean_accuracy)
  avg_acc.append(mean_accuracy)

print(avg_acc)

In [ ]:
# test different embedding dimensions

results = []

def dim_10():
  embedding_dim = 10
  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

def dim_100():
  embedding_dim = 100
  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

def dim_300():
  embedding_dim = 300
  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

models = [dim_10, dim_100, dim_300]
avg_acc = []
tests = 10

for model_func in models:

  total_accuracy = 0

  for i in range(tests):
    model = model_func()
    training = model.fit(X_train, y_train, epochs=5, verbose=True, validation_split=0.1, batch_size=50)
    accuracy, cm, cf = eval_model(model)

    print(accuracy)

    total_accuracy += accuracy

  mean_accuracy = total_accuracy / tests
  print('MEAN ACCURACY -', mean_accuracy)
  avg_acc.append(mean_accuracy)

print(avg_acc)

# Train Model

In [14]:
# pre processing the data for input to the embedding layer
def process_data():
  # split the data
  reviews_train, reviews_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.20)

  # tokenize the dataset
  tokenizer = Tokenizer(num_words=43297)
  tokenizer.fit_on_texts(reviews_train)
  X_train = tokenizer.texts_to_sequences(reviews_train)
  X_test = tokenizer.texts_to_sequences(reviews_test)

  vocab_size = len(tokenizer.word_index) + 1

  # the max length of a reviews, parameter for the embedding layer
  maxlen = max([len(x) for x in X_train] + [len(x) for x in X_test])

  # pad the tokenized data with 0s
  X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
  X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

  return (vocab_size, maxlen, X_train, X_test, y_train, y_test)

vocab_size, maxlen, X_train, X_test, y_train, y_test = process_data()
print(vocab_size, maxlen, len(X_train))

39386 2392 1600


In [15]:
# produces a confusion matrix and determines Precision, Recall, Accuracy and F1-score of model

def eval_model(model):
  loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
  print("Testing Accuracy: {:.4f}".format(accuracy))

  # predict test data using model
  y_pred = model.predict(X_test, batch_size=50, verbose=1)
  y_pred_bool = []

  # convert probablities to classes
  for p in y_pred:
    if p >= 0.5:
      y_pred_bool.append(1)
    else:
      y_pred_bool.append(0)

  # print results
  cm = confusion_matrix(y_test, y_pred_bool)
  cf = classification_report(y_test, y_pred_bool)

  print('\t  Predicted')
  print('\t   0    1  ')
  print(f'\t0 {cm[0]}')
  print('True\t')
  print(f'\t1 {cm[1]}')

  print(cf)

  return (accuracy, cm, cf)

In [16]:
print(X_train[568])

[1768 4753  643 ...    0    0    0]


In [17]:
# early stopping callbacks
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='loss', patience=3)

In [18]:
# -- define the model --

def lstm_classifier():
  embedding_dim = 300

  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.LSTM(4, return_sequences=True, dropout=0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  model.summary()

  return model

In [19]:
# -- training --

model2 = lstm_classifier()
training = model2.fit(X_train, y_train, epochs=5, verbose=True, validation_split=0.1, batch_size=50, callbacks=[earlystop])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2392, 300)         11815800  
                                                                 
 lstm (LSTM)                 (None, 2392, 4)           4880      
                                                                 
 lstm_1 (LSTM)               (None, 2392, 4)           144       
                                                                 
 flatten (Flatten)           (None, 9568)              0         
                                                                 
 dense (Dense)               (None, 100)               956900    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 12,777,825
Trainable params: 12,777,825
No

In [20]:
eval_model(model2)

Testing Accuracy: 0.8175
8/8 [==============================] - 4s 420ms/step
	  Predicted
	   0    1  
	0 [191  21]
True	
	1 [ 52 136]
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       212
           1       0.87      0.72      0.79       188

    accuracy                           0.82       400
   macro avg       0.83      0.81      0.81       400
weighted avg       0.82      0.82      0.82       400



(0.8174999952316284, array([[191,  21],
        [ 52, 136]]), '              precision    recall  f1-score   support\n\n           0       0.79      0.90      0.84       212\n           1       0.87      0.72      0.79       188\n\n    accuracy                           0.82       400\n   macro avg       0.83      0.81      0.81       400\nweighted avg       0.82      0.82      0.82       400\n')